In [4]:
import pandas as pd

cab_rides = pd.read_csv("cab_rides.csv/cab_rides.csv")
cab_rides.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [9]:
uber_rides_data = cab_rides.loc[cab_rides["cab_type"] == "Uber"]
uber_rides_data.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
12,1.11,Uber,1543615981179,West End,North End,12.0,1.0,009e9c53-074d-43cf-aef2-0fbc7a47ed3d,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,UberXL
13,1.11,Uber,1544698211014,West End,North End,16.0,1.0,23f145da-f0c1-4d1f-a184-496bc003a7db,6c84fd89-3f11-4782-9b50-97c468b19529,Black
14,1.11,Uber,1544728503935,West End,North End,7.5,1.0,357559cb-8c58-4278-a41a-e33b2e0997a3,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX
15,1.11,Uber,1545004511430,West End,North End,7.5,1.0,50ef1165-9d23-416c-a65c-18906207b295,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,WAV
16,1.11,Uber,1544748007961,West End,North End,26.0,1.0,91c4861c-1780-42b0-bca1-bbd64a422cc3,6d318bcc-22a3-4af6-bddd-b409bfce1546,Black SUV


In [10]:
uber_rides_data["name"].unique()

array(['UberXL', 'Black', 'UberX', 'WAV', 'Black SUV', 'UberPool', 'Taxi'],
      dtype=object)

In [19]:
uber_rides = uber_rides_data[["distance", "source", "destination", "price", "time_stamp", "name"]]
uber_rides

,distance,source,destination,price,time_stamp,name
12,1.11,North End,West End,12.0,1543615981179,UberXL
13,1.11,North End,West End,16.0,1544698211014,Black
14,1.11,North End,West End,7.5,1544728503935,UberX
15,1.11,North End,West End,7.5,1545004511430,WAV
16,1.11,North End,West End,26.0,1544748007961,Black SUV
...,...,...,...,...,...,...
693066,1.00,West End,North End,13.0,1543708385534,UberXL
693067,1.00,West End,North End,9.5,1543708385534,UberX
693068,1.00,West End,North End,NaN,1543708385534,Taxi
693069,1.00,West End,North End,27.0,1543708385534,Black SUV


In [17]:
uber_rides_data["destination"].nunique()

12

In [18]:
uber_rides_data["source"].nunique()

12